## Test the Retrieval Latency of Approximate vs Exact Matching 

In [ ]:
import tensorflow as tf
import time

In [ ]:
PROJECT_ID = 'ksalama-cloudml'
BUCKET = 'ksalama-cloudml'
INDEX_DIR = f'gs://{BUCKET}/bqml/scann_index'
BQML_MODEL_DIR = f'gs://{BUCKET}/bqml/item_matching_model'
LOOKUP_MODEL_DIR = f'gs://{BUCKET}/bqml/embedding_lookup_model'

In [ ]:
songs = {
    '2114406': 'Metallica: Nothing Else Matters',
    '2114402': 'Metallica: The Unforgiven',
    '2120788': 'Limp Bizkit: My Way',
    '2120786': 'Limp Bizkit: My Generation',
    '1086322': 'Jacques Brel: Ne Me Quitte Pas',
    '3129954': 'Édith Piaf: Non, Je Ne Regrette Rien',
    '53448': 'France Gall: Ella, Elle l\'a',
    '887688': 'Enrique Iglesias: Tired Of Being Sorry',
    '562487': 'Shakira: Hips Don\'t Lie',
    '833391': 'Ricky Martin: Livin\' la Vida Loca',
    '1098069': 'Snoop Dogg: Drop It Like It\'s Hot',
    '910683': '2Pac: California Love',
    '1579481': 'Dr. Dre: The Next Episode',
    '2675403': 'Eminem: Lose Yourself',
    '2954929': 'Black Sabbath: Iron Man',
    '625169': 'Black Sabbath: Paranoid',
}

## Exact Matching

In [ ]:
class ExactMatcher(object):
  def __init__(self, model_dir):
    print("Loading exact matchg model...")
    self.model = tf.saved_model.load(model_dir)
    print("Exact matchg model is loaded.")
  
  def match(self, instances):
    outputs = self.model.signatures['serving_default'](tf.constant(instances, tf.dtypes.int64))
    return outputs['predicted_item2_Id'].numpy()

In [ ]:
exact_matcher = ExactMatcher(BQML_MODEL_DIR)

In [ ]:
exact_matches = {}

start_time = time.time()
for i in range(100):
  for song in songs:
    matches = exact_matcher.match([int(song)])
    exact_matches[song] = matches.tolist()[0]
end_time = time.time()
exact_elapsed_time = end_time - start_time

print(f'Elapsed time: {round(exact_elapsed_time, 3)} seconds - average time: {exact_elapsed_time / (100 * len(songs))} seconds')

## Approximate Matching (ScaNN)

In [ ]:
from index_server.matching import ScaNNMatcher
scann_matcher = ScaNNMatcher(INDEX_DIR)
embedding_lookup = tf.saved_model.load(LOOKUP_MODEL_DIR)

In [ ]:
approx_matches = dict()

start_time = time.time()
for i in range(100):
  for song in songs:
    vector = embedding_lookup([song]).numpy()[0]
    matches = scann_matcher.match(vector, 50)
    approx_matches[song] = matches
end_time = time.time()
scann_elapsed_time = end_time - start_time

print(f'Elapsed time: {round(scann_elapsed_time, 3)} seconds - average time: {scann_elapsed_time / (100 * len(songs))} seconds')

In [ ]:
speedup_percent = round(exact_elapsed_time / scann_elapsed_time, 1)
print(f'ScaNN speedup: {speedup_percent}x')

## License

Copyright 2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at: http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. 

See the License for the specific language governing permissions and limitations under the License.

**This is not an official Google product but sample code provided for an educational purpose**